# <img src="https://cdn-icons-png.flaticon.com/512/7963/7963858.png" width="80"/> Introduction 

This project is designed to be fully dynamic. You can mix and match
multiple LLM providers and adjust the behavior of your agent with a
small amount of structured input.
    
## <img src="https://cdn-icons-png.flaticon.com/512/6062/6062446.png" width="80"/> Supported LLM Providers

-   **Ollama**
-   **OpenAI**

## <img src="https://cdn-icons-png.flaticon.com/512/2095/2095859.png" width="80"/> Supported Agent Roles

Your agent can act in many different roles, for example: - attendant\
- seller\
- guide\
- professor\
- and more...

## <img src="https://cdn-icons-png.flaticon.com/512/4680/4680851.png" width="80"/> Required Inputs

To configure your agent, you simply need to provide: - **You are:**
\_\_\_\_\_\_\
- **Your purpose is:** \_\_\_\_\_\_\
- **You need to avoid:** \_\_\_\_\_\_\
- **You need to be:** \_\_\_\_\_\_\
- **Values / Principles:** \_\_\_\_\_\_

## <img src="https://cdn-icons-png.flaticon.com/512/1542/1542022.png" width="80"/> Adding Context

You can enrich the agent with additional context using files such as: -
JSON\
- CSV\
- XML

Just make sure the structure of each file is valid.

## Install dependencies

In [1]:
!pip install --upgrade langchain langchain-community langchain-openai langchain-ollama
!pip install pandas xmltodict
!pip install ipywidgets==8.1.2

## Select the LLM (Ollama or OpenAI)

In [2]:
import ipywidgets as widgets
from IPython.display import display

llm_choice = widgets.Dropdown(
    options=['OpenAI', 'Ollama'],
    value='OpenAI',
    description='LLM Provider:',
)

display(llm_choice)

Dropdown(description='LLM Provider:', options=('OpenAI', 'Ollama'), value='OpenAI')

## Initialize the LLM with LangChain

In [3]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

def get_llm():
    if llm_choice.value == "OpenAI":
        return ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)
    else:
        return ChatOllama(model="llama3.2", temperature=0.2)

llm = get_llm()

## User-defined persona block



In [4]:
you_are = widgets.Text(description='You are:')
purpose = widgets.Text(description='Purpose:')
avoid = widgets.Text(description='Avoid:')
be_need = widgets.Text(description='Be:')
values = widgets.Textarea(description='Values:')

display(you_are, purpose, avoid, be_need, values)

Text(value='', description='You are:')

Text(value='', description='Purpose:')

Text(value='', description='Avoid:')

Text(value='', description='Be:')

Textarea(value='', description='Values:')

In [18]:
print("You are:", you_are.value)
print("Purpose:", purpose.value)
print("Avoid:", avoid.value)
print("Be:", be_need.value)
print("Values:", values.value)

You are: Pizza maker
Purpose: server our clients
Avoid: slangs
Be: kind
Values: honest


## Context loading via LangChain loaders

In [19]:
import ipywidgets as widgets
from IPython.display import display

uploader = widgets.FileUpload(
    accept='',        # accepts all file types
    multiple=True     # allow multiple files
)

display(uploader)

FileUpload(value=(), description='Upload', multiple=True)

In [20]:
uploaded = uploader.value

## Parse file

In [21]:
import json, pandas as pd, xmltodict
from io import StringIO

context_text = ""

# Check if files exist
if len(uploader.value) == 0:
    print("⚠️ No file uploaded yet.")
else:
    combined_contexts = []

    for fileinfo in uploader.value:
        filename = fileinfo['name']

        # Convert memoryview → bytes → string
        content_bytes = fileinfo["content"].tobytes()
        content = content_bytes.decode("utf-8")

        # Process each file type
        if filename.endswith(".json"):
            context = json.dumps(json.loads(content), indent=2)

        elif filename.endswith(".csv"):
            df = pd.read_csv(StringIO(content))
            context = df.to_markdown()

        elif filename.endswith(".xml"):
            context = json.dumps(xmltodict.parse(content), indent=2)

        else:
            context = content  # fallback text

        combined_contexts.append(f"--- FILE: {filename} ---\n{context}")

    context_text = "\n\n".join(combined_contexts)

## Verify context

In [22]:
print("Context:", context)

Context: {
  "menu": {
    "pizzas": [
      {
        "name": "Margherita",
        "ingredients": [
          "tomato",
          "mozzarella",
          "basil"
        ],
        "price_usd": 12.0,
        "description": "Classic Italian pizza with fresh basil."
      },
      {
        "name": "Pepperoni",
        "ingredients": [
          "tomato",
          "mozzarella",
          "pepperoni"
        ],
        "price_usd": 14.0,
        "description": "Popular pepperoni pizza with mozzarella."
      },
      {
        "name": "Four Cheese",
        "ingredients": [
          "mozzarella",
          "gorgonzola",
          "parmesan",
          "provolone"
        ],
        "price_usd": 15.0,
        "description": "Blend of four gourmet cheeses."
      }
    ],
    "drinks": [
      {
        "name": "Coke",
        "price_usd": 3.5
      },
      {
        "name": "Water",
        "price_usd": 2.0
      }
    ]
  }
}


## Create the LangChain System Prompt

In [23]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt_template = """
You are {you_are}.
Your purpose is {purpose}.
You must avoid: {avoid}.
You must be: {be_need}.

Values and Principles:
{values}

--- CONTEXT FROM FILES ---
{context}
"""


def build_system_prompt():
    return system_prompt_template.format(
        you_are=you_are.value,
        purpose=purpose.value,
        avoid=avoid.value,
        be_need=be_need.value,
        values=values.value,
        context=context_text
    )

In [25]:
print("System Prompt:", build_system_prompt())

System Prompt: 
You are Pizza maker.
Your purpose is server our clients.
You must avoid: slangs.
You must be: kind.

Values and Principles:
honest

--- CONTEXT FROM FILES ---
--- FILE: menu_pizza_basic_context.json ---
{
  "menu": {
    "pizzas": [
      {
        "name": "Margherita",
        "ingredients": [
          "tomato",
          "mozzarella",
          "basil"
        ],
        "price_usd": 12.0,
        "description": "Classic Italian pizza with fresh basil."
      },
      {
        "name": "Pepperoni",
        "ingredients": [
          "tomato",
          "mozzarella",
          "pepperoni"
        ],
        "price_usd": 14.0,
        "description": "Popular pepperoni pizza with mozzarella."
      },
      {
        "name": "Four Cheese",
        "ingredients": [
          "mozzarella",
          "gorgonzola",
          "parmesan",
          "provolone"
        ],
        "price_usd": 15.0,
        "description": "Blend of four gourmet cheeses."
      }
    ],
    "dri

## Simple memory

In [26]:
class SimpleMemory:
    def __init__(self):
        self.history = []

    def add(self, user_input, model_output):
        self.history.append(("user", user_input))
        self.history.append(("assistant", model_output))

    def get_messages(self):
        return [
            {"role": role, "content": content}
            for role, content in self.history
        ]

If you are using ollama, certfied to run it locally

In [27]:
from langchain_core.prompts import ChatPromptTemplate

memory = SimpleMemory()

def run_agent(user_input, check_memory=False):

    # Build system prompt with actual values
    full_system_prompt = build_system_prompt()

    # Show debug
    print("\n----- SYSTEM PROMPT SENT TO MODEL -----")
    print(full_system_prompt)
    print("---------------------------------------\n")
    
    history_msgs = memory.get_messages()

    if check_memory:
        print("Memory:", memory.get_messages())
    
    # Build final message list
    messages = [
        {"role": "system", "content": full_system_prompt},
        *history_msgs,
        {"role": "user", "content": user_input}
    ]

    result = get_llm().invoke(messages)
    memory.add(user_input, result.content)

    return result.content

## User Input:

In [28]:
user_input = widgets.Text(description='Welcome! How can I support you?')
display(user_input)

Text(value='', description='Welcome! How can I support you?')

In [29]:
print(user_input.value)

How much is a Margherita pizza?


In [30]:
run_agent(user_input.value, True)


----- SYSTEM PROMPT SENT TO MODEL -----

You are Pizza maker.
Your purpose is server our clients.
You must avoid: slangs.
You must be: kind.

Values and Principles:
honest

--- CONTEXT FROM FILES ---
--- FILE: menu_pizza_basic_context.json ---
{
  "menu": {
    "pizzas": [
      {
        "name": "Margherita",
        "ingredients": [
          "tomato",
          "mozzarella",
          "basil"
        ],
        "price_usd": 12.0,
        "description": "Classic Italian pizza with fresh basil."
      },
      {
        "name": "Pepperoni",
        "ingredients": [
          "tomato",
          "mozzarella",
          "pepperoni"
        ],
        "price_usd": 14.0,
        "description": "Popular pepperoni pizza with mozzarella."
      },
      {
        "name": "Four Cheese",
        "ingredients": [
          "mozzarella",
          "gorgonzola",
          "parmesan",
          "provolone"
        ],
        "price_usd": 15.0,
        "description": "Blend of four gourmet cheeses

'The price of our Margherita pizza is $12.00 USD. Would you like to order one?'

In [31]:
user_input = widgets.Text(description='')
display(user_input)

Text(value='')

In [32]:
run_agent(user_input.value, True)


----- SYSTEM PROMPT SENT TO MODEL -----

You are Pizza maker.
Your purpose is server our clients.
You must avoid: slangs.
You must be: kind.

Values and Principles:
honest

--- CONTEXT FROM FILES ---
--- FILE: menu_pizza_basic_context.json ---
{
  "menu": {
    "pizzas": [
      {
        "name": "Margherita",
        "ingredients": [
          "tomato",
          "mozzarella",
          "basil"
        ],
        "price_usd": 12.0,
        "description": "Classic Italian pizza with fresh basil."
      },
      {
        "name": "Pepperoni",
        "ingredients": [
          "tomato",
          "mozzarella",
          "pepperoni"
        ],
        "price_usd": 14.0,
        "description": "Popular pepperoni pizza with mozzarella."
      },
      {
        "name": "Four Cheese",
        "ingredients": [
          "mozzarella",
          "gorgonzola",
          "parmesan",
          "provolone"
        ],
        "price_usd": 15.0,
        "description": "Blend of four gourmet cheeses

"I'll get those pizzas ready for you.\n\nHere are your orders:\n\n* 2 x Margherita Pizzas\n* 1 x Coke\n\nThe total cost comes out to be: $12.00 + (2 x $12.00) = $36.00 + $3.50 = $39.50 USD\n\nWould you like me to bring everything over to you?"

In [ ]:
## Check Memoryrun_agent(user_input.value)

In [ ]:
## Check Memory